# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls 

In [2]:
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [3]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

# reshape to 2d feature
train = train.reshape(-1, 1, 11, 39)
test = test.reshape(-1, 1, 11, 39)

Loading data ...


In [4]:
print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Size of training data: (1229932, 1, 11, 39)
Size of testing data: (451552, 1, 11, 39)


## Create Dataset

In [5]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()

        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
        

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)



Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [6]:
VAL_RATIO=0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 1, 11, 39)
Size of validation set: (245987, 1, 11, 39)


## Normalization

In [7]:
tr = train_x[:, 0, 5].reshape(-1, 1, 1, 39)
mean = tr.mean(axis=0, keepdims=True)
std = tr.std(axis=0, keepdims=True)
norm = lambda x: (x-mean)/std
train_x = norm(train_x)
val_x = norm(val_x)
test = norm(test)

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [8]:
BATCH_SIZE = 256

from torch.utils.data import DataLoader


train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [9]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

252

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [10]:
import torch
import torch.nn as nn

In [11]:
class BlockConv2d(nn.Module):
    def __init__(self, ch_in, ch_out, k, act=None, use_bn=True, drop=0):
        super(BlockConv2d, self).__init__()
        list = [
            nn.Conv2d(ch_in, ch_out, k),
        ]
        if use_bn: list.append(nn.BatchNorm2d(ch_out))
        if act: list.append(act)
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)

class BlockLinear(nn.Module):
    def __init__(self, ch_in, ch_out, act=None, use_bn=True, drop=0):
        super(BlockLinear, self).__init__()
        list = [
            nn.Linear(ch_in, ch_out)
        ]
        if use_bn: list.append(nn.BatchNorm1d(ch_out))
        if act: list.append(act)
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)


In [12]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.act = nn.SELU()
        # self.act = nn.ReLU()

        # size = (1, 11, 39)
        self.n_0 = [1] + [1024, 256] + [256]
        self.k_0 = [(1, 39)] + [(1, 1)] + [(9, 1)]
        self.l_0 = len(self.k_0)
        self.conv0 = [
            BlockConv2d(self.n_0[i], self.n_0[i+1], self.k_0[i], self.act, use_bn=True, drop=0.35)
            for i in range(self.l_0)]
        self.conv0 = nn.Sequential(*self.conv0)

        # size = (128, 9, 3)
        self.flat = nn.Flatten()

        # size = (9*9*3)
        self.n_1 = [3*256] + [512, 256, 128]
        self.l_1 = len(self.n_1)
        self.linears = [
            BlockLinear(self.n_1[i], self.n_1[i+1], self.act, use_bn=True, drop=0.15)
            for i in range(self.l_1-1)
        ]
        self.linears = nn.Sequential(*self.linears)

        self.out = nn.Linear(self.n_1[-1], 39)

    def forward(self, x):
        x = self.conv0(x)
        x = self.flat(x)
        x = self.linears(x)
        x = self.out(x)
        return x

## Training

In [13]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [14]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [15]:
# fix random seed for reproducibility
same_seeds(52728)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 200               # number of training epoch
learning_rate = 1e-4       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'
overfit_model_path = './overfit_model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
print(model.load_state_dict(torch.load(model_path)))
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

DEVICE: cuda
<All keys matched successfully>


In [16]:
print(model)

Classifier(
  (act): SELU()
  (conv0): Sequential(
    (0): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(1, 1024, kernel_size=(1, 39), stride=(1, 1))
        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SELU()
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (1): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SELU()
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (2): BlockConv2d(
      (net): Sequential(
        (0): Conv2d(256, 256, kernel_size=(9, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SELU()
        (3): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linears): Sequential(
    (0): BlockLinear(
    

In [17]:
gain = 3/4 # for selu
for n, p in model.named_parameters():
    print(n, p.shape)
    if p.dim() >= 2:
        print("normal init")
        nn.init.xavier_normal_(p, gain)

conv0.0.net.0.weight torch.Size([1024, 1, 1, 39])
normal init
conv0.0.net.0.bias torch.Size([1024])
conv0.0.net.1.weight torch.Size([1024])
conv0.0.net.1.bias torch.Size([1024])
conv0.1.net.0.weight torch.Size([256, 1024, 1, 1])
normal init
conv0.1.net.0.bias torch.Size([256])
conv0.1.net.1.weight torch.Size([256])
conv0.1.net.1.bias torch.Size([256])
conv0.2.net.0.weight torch.Size([256, 256, 9, 1])
normal init
conv0.2.net.0.bias torch.Size([256])
conv0.2.net.1.weight torch.Size([256])
conv0.2.net.1.bias torch.Size([256])
linears.0.net.0.weight torch.Size([512, 768])
normal init
linears.0.net.0.bias torch.Size([512])
linears.0.net.1.weight torch.Size([512])
linears.0.net.1.bias torch.Size([512])
linears.1.net.0.weight torch.Size([256, 512])
normal init
linears.1.net.0.bias torch.Size([256])
linears.1.net.1.weight torch.Size([256])
linears.1.net.1.bias torch.Size([256])
linears.2.net.0.weight torch.Size([128, 256])
normal init
linears.2.net.0.bias torch.Size([128])
linears.2.net.1.weig

In [31]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')
else:
    torch.save(model.state_dict(), overfit_model_path)
    print('saving overfit model')

  0%|          | 1/3844 [00:00<07:06,  9.02it/s][001/200] Train Acc: 0.810231 Loss: 0.559422 | Val Acc: 0.769061 loss: 0.726138
saving model with acc 0.769
  0%|          | 1/3844 [00:00<07:48,  8.21it/s][002/200] Train Acc: 0.813012 Loss: 0.550973 | Val Acc: 0.770110 loss: 0.729367
saving model with acc 0.770
  0%|          | 1/3844 [00:00<07:02,  9.10it/s][004/200] Train Acc: 0.785211 Loss: 0.635318 | Val Acc: 0.771049 loss: 0.699930
saving model with acc 0.771
  0%|          | 1/3844 [00:00<06:27,  9.91it/s][007/200] Train Acc: 0.785982 Loss: 0.630941 | Val Acc: 0.771338 loss: 0.699686
saving model with acc 0.771
 39%|███▉      | 1503/3844 [00:36<00:57, 40.80it/s]


KeyboardInterrupt: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [16]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [17]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [18]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))